In [1]:
import pandas as pd

In [2]:
XML_PATH = 'data/data-20251201T0000-structure-20220404T0000.xml'

In [3]:
df = pd.read_xml(XML_PATH)

In [4]:
dfd = df[df['status'] == 'действующая']
len(dfd)

5125

In [5]:
dfd = dfd[dfd['licensed_activity'] == 'Радиовещание радиоканала']
len(dfd)

3513

In [6]:
dfd['org_name'].unique()

array(['ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "РАДИО НОСТАЛЬЖИ - ОРЕНБУРГ"',
       'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ТЕЛЕРАДИОКОМПАНИЯ "ЮГ"',
       'Общество с ограниченной ответственностью "Полярная звезда"', ...,
       'МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "МУЖИ-ИНФОРМ"',
       'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "СТАРСМЕДИА"',
       'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "РАДИОАКТИВНОСТЬ"'],
      shape=(1703,), dtype=object)

In [7]:
import xml.etree.ElementTree as ET
from collections import defaultdict


# пространство имён из XML
NS = {"rkn": "http://rsoc.ru/opendata/7705846236-LicBroadcast" }  # если другое — поправим

tree = ET.parse(XML_PATH)
root = tree.getroot()

audience = defaultdict(float)

In [8]:
for rec in root.findall(".//rkn:record", NS):
    status = rec.findtext("rkn:status", "", NS).strip()
    lic = rec.findtext("rkn:licensed_activity", "", NS).strip()

    if status != "действующая":
        continue
    if lic != "Радиовещание радиоканала":
        continue

    org = rec.findtext("rkn:org_name", "", NS).strip()
    if not org:
        continue

    # перебор всех строк grid
    for row in rec.findall(".//rkn:grid/rkn:row", NS):
        pop_text = row.findtext("rkn:population", "0", NS).strip()

        # формат чисел в данных: "60,0"
        pop_text = pop_text.replace(",", ".")
        try:
            pop = float(pop_text)
        except ValueError:
            pop = 0.0

        audience[org] += pop

In [9]:
sorted_audience = dict(sorted(audience.items(), key=lambda item: item[1], reverse=True))

In [10]:
# вывод результата
with open('data/orgs.txt', 'w', encoding='utf-8') as f:
    for org_name, total_population in sorted_audience.items():
        f.write(f"{org_name}: {total_population}\n")